In [55]:
import pandas as pd
import model
import datetime
import matplotlib.pyplot as plt
import networkx as nx
from importlib import reload
import numpy as np
import io
import matplotlib.pyplot as plt
import json
from time import time

In [17]:
import utils

In [18]:
%matplotlib inline

In [19]:
def data():
    datatable_csv = """case_id,event,timestamp,user
1111,written,15/07/2016,marchand
1111,sent,16/07/2016,marchand
1111,received,17/07/2016,rioust
2222,written,18/07/2016,blasco
2222,sent,19/07/2016,blasco
2222,received,20/07/2016,burel
2222,forwarded,21/07/2016,burel
2222,received,22/07/2016,rioust
3333,written,23/07/2016,blasco
3333,deleted,24/07/2016,blasco
4444,written,25/07/2016,rioust
4444,sent,26/07/2016,rioust
4444,received,27/07/2016,marchand
4444,received,28/07/2016,burel
4444,received,29/07/2016,blasco"""
    return pd.read_csv(io.StringIO(datatable_csv), delimiter=',').set_index("case_id", drop=False)

In [20]:
datatable = data()
event_log = pd.read_csv("../../data/event_log.csv")

In [741]:
reload(model)
model.PandasProcessModel.set_process_data(event_log)
ppm = model.PandasProcessModel("Event log")

In [743]:
%%time
for i in range(10):
    a = ppm.get_graph_definition()

Wall time: 2.59 s


In [617]:
filtered_process_data = ppm.get_filtered_process_data()

In [618]:
%%time
filtered_process_data.reset_index(inplace=True, drop=True)

Wall time: 0 ns


In [619]:
%%time
concat_table = pd.concat([filtered_process_data.iloc[:-1][["case_id", "event"]],
                          filtered_process_data.iloc[1:][["case_id", "event"]]],
                         axis=1)

Wall time: 93 ms


In [626]:
event_log[["case_id", "event"]].head()

,case_id,event
case_id,,
1,1,Create PO
1,1,Goods receipt
1,1,Invoice - gross
1,1,No Hold
2,2,PO Requisition


In [625]:
event_log[["case_id", "event"]].rename(columns = {"case_id": "case_id_new"}).head()

,case_id_new,event
case_id,,
1,1,Create PO
1,1,Goods receipt
1,1,Invoice - gross
1,1,No Hold
2,2,PO Requisition


In [613]:
concat_table = pd.concat([
        pd.DataFrame(filtered_process_data.iloc[:-1][["case_id", "event"]].values,
                     columns=["case_id_source", "event_source"]),
        pd.DataFrame(filtered_process_data.iloc[1:].values,
                     columns=filtered_process_data.columns.values)
    ],
        axis=1)
    print("Concat table created in", time() - t)
    concat_table = concat_table[concat_table["case_id_source"] == concat_table["case_id"]]
    concat_table.drop("case_id_source", axis=1).set_index("case_id", inplace=True, drop=False)
    print("Concat table created in", time() - t)
    t = time()
    grouped = concat_table.drop("timestamp", inplace=False, axis=1).groupby(["event_source", "event"])
    aggregated = grouped.aggregate({
        "case_id": (lambda x: x.nunique()),
        "user": (lambda x: x.nunique())
    })

Concat table created in 0.1380000114440918
Wall time: 236 ms


In [607]:
%%time
a =ppm.get_graph_definition()

Nodes created in 0.03500008583068848
Concat table created in 1.8480000495910645
Concat table created in 2.0490000247955322
Aggregated table created in 0.15899991989135742
Edges created in 0.003000020980834961
Wall time: 2.3 s


In [595]:
event_log.head()

,case_id,event,timestamp,user,timestamp_day,timestamp_day_integers
case_id,,,,,,
1,1,Create PO,2015-05-11 00:00:00,TISSOTBE,2015-05-11,1431302400000
1,1,Goods receipt,2015-05-20 10:41:25,BENEDICTE TISSOT,2015-05-20,1432080000000
1,1,Invoice - gross,2015-06-30 09:04:29,ALBUQUEL,2015-06-30,1435622400000
1,1,No Hold,2015-06-30 09:04:34,NOBODY,2015-06-30,1435622400000
2,2,PO Requisition,2015-06-29 23:59:55,LEPINAMA,2015-06-29,1435536000000


In [596]:
grouped = event_log.groupby("user")

In [678]:
grouped.

AttributeError: type object 'DataFrameGroupBy' has no attribute 'nunique'

In [676]:
type(grouped).count

<function pandas.core.groupby.DataFrameGroupBy.count>

In [682]:
len

<function len>

In [710]:
%%time
for i in range(100):
    a = grouped.aggregate({
            "case_id": len,
            "user": np.count_nonzero
        })

Wall time: 3.42 s


In [711]:
%%time
for i in range(100):
    a = pd.concat([grouped.case_id.count(), grouped.user.nunique()], axis=1)

Wall time: 3.85 s


In [597]:
%%time
a = grouped.aggregate({
        "case_id": lambda x: float(len(x))
    })

Wall time: 82 ms


In [599]:
%%time
a = grouped.case_id.nunique()

Wall time: 51 ms


In [600]:
a.head()

user
A.DUMAS         9
A.WARDOWSKI     3
ABADIENI        1
AGNES CREVAT    3
AIME BRIQUET    3
Name: case_id, dtype: int64

In [570]:
ppm.get_graph_definition()["graph_edges"]

[('Approve PO', 'Goods receipt', {'case_id': 10.0, 'user': 2.0}),
 ('Approve PO', 'Invoice - gross', {'case_id': 3.0, 'user': 1.0}),
 ('Approve PO', 'PO Requisition', {'case_id': 3.0, 'user': 1.0}),
 ('Create PO', 'Approve PO', {'case_id': 7.0, 'user': 1.0}),
 ('Create PO', 'Goods receipt', {'case_id': 22908.0, 'user': 6.0}),
 ('Create PO', 'Invoice - gross', {'case_id': 57.0, 'user': 1.0}),
 ('Goods receipt', 'Invoice - gross', {'case_id': 22914.0, 'user': 1.0}),
 ('Goods receipt', 'PO Requisition', {'case_id': 1.0, 'user': 1.0}),
 ('Invoice - gross', 'Manual Release', {'case_id': 6.0, 'user': 1.0}),
 ('Invoice - gross', 'No Hold', {'case_id': 22968.0, 'user': 1.0}),
 ('No Hold', 'Goods receipt', {'case_id': 42.0, 'user': 3.0}),
 ('No Hold', 'PO Requisition', {'case_id': 5.0, 'user': 2.0}),
 ('PO Requisition', 'Create PO', {'case_id': 31.0, 'user': 3.0})]

In [555]:
f = model.PandasListFilter("user", ["ALBUQUEL"])

In [568]:
time()

1471006690.339293

In [567]:
a = time()

In [563]:
%%time
for i in range(100):
    ppm.add_filter(f)

Wall time: 4.65 s


In [554]:
%%time
a = ppm.get_daily_barchart()

Wall time: 55 ms


In [349]:
%%time
a = ppm.get_aggregated_data("timestamp_day_integers", "timestamp", "count")

Wall time: 52.1 ms


In [54]:
t = time()
b = ppm.get_piechart("user")
print(time()-t)

0.09800553321838379


In [564]:
a = 5623457653246148716329121238763

In [36]:
%%time
c = ppm.get_piechart("event")

Wall time: 71 ms


In [320]:
indexed_event_log = ppm.get_process_data()

In [404]:
ma_liste = [event_log for x in range(10)]

In [410]:
big_table = pd.concat(ma_liste)

In [411]:
big_table.shape

(2225510, 4)

In [441]:
%%time
a = big_table.case_id.isin([200,201,202,203,204,205,206,207])

Wall time: 52 ms


In [448]:
%%time
a = np.exp(big_table.case_id) 
print(type(a))

<class 'pandas.core.series.Series'>
Wall time: 255 ms


In [449]:
%%time
a = big_table.case_id.map(lambda x : np.exp(x))

Wall time: 4.26 s


In [421]:
%%time
a = big_table.case_id.map(lambda x: x>200)

Wall time: 1.34 s


In [483]:
%%timeit
new_index = set(indexed_event_log[indexed_event_log[filter.filter_by].isin(['BENEDICTE TISSOT'])].index)

10 loops, best of 3: 20.1 ms per loop


In [450]:
filter.filter_list

'BENEDICTE TISSOT'

In [484]:
%%timeit
new_index2 = indexed_event_log.case_id.isin(
    indexed_event_log[indexed_event_log[filter.filter_by].isin(['BENEDICTE TISSOT'])].index)

10 loops, best of 3: 50 ms per loop


In [358]:
new_index2

case_id
1         True
1         True
1         True
1         True
2        False
2        False
2        False
2        False
2        False
2        False
3         True
3         True
3         True
3         True
4         True
4         True
4         True
4         True
5        False
5        False
5        False
5        False
6        False
6        False
6        False
6        False
6        False
6        False
7         True
7         True
         ...  
52564     True
52564     True
52565     True
52565     True
52565     True
52565     True
52566     True
52566     True
52566     True
52566     True
52567     True
52567     True
52567     True
52567     True
52568    False
52568    False
52568    False
52568    False
52569     True
52569     True
52569     True
52569     True
52570     True
52570     True
52570     True
52570     True
52571    False
52571    False
52571    False
52571    False
Name: case_id, dtype: bool

In [292]:
%%time
c = functools.reduce(operator.and_, [])

TypeError: reduce() of empty sequence with no initial value

In [259]:
%%time
c = new_conjunction([a,b])

TypeError: reduce() takes no keyword arguments

In [217]:
d = np.ones(1000000000, dtype=bool)

In [251]:
def new_conjunction(lst):
    return functools.reduce(np.logical_and, [x for x in lst], i)

In [284]:
%%time
len([a,b]) < 1

Wall time: 0 ns


False

In [277]:
%%time
c = functools.reduce(np.logical_and, [a,b], d)

Wall time: 1.27 s


In [250]:
new_conjunction([])

TypeError: reduce() of empty sequence with no initial value

In [229]:
import functools

In [241]:
%%time
c = new_conjunction([a,b])

Wall time: 732 ms


In [226]:
ppm.get_process_data().user.map(filter.filtering_function)

case_id
1        False
1         True
1        False
1        False
2        False
2        False
2        False
2        False
2        False
2        False
3        False
3         True
3        False
3        False
4        False
4         True
4        False
4        False
5        False
5        False
5        False
5        False
6        False
6        False
6        False
6        False
6        False
6        False
7        False
7         True
         ...  
52564    False
52564    False
52565    False
52565     True
52565    False
52565    False
52566    False
52566     True
52566    False
52566    False
52567    False
52567     True
52567    False
52567    False
52568    False
52568    False
52568    False
52568    False
52569    False
52569     True
52569    False
52569    False
52570    False
52570     True
52570    False
52570    False
52571    False
52571    False
52571    False
52571    False
Name: user, dtype: bool

In [218]:
%%time
c = utils.conjunction([a,b], d)

Traceback (most recent call last):
  File "C:\Users\jblasco\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2885, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-218-6ecd45cb6446>", line 1, in <module>
    get_ipython().run_cell_magic('time', '', 'c = utils.conjunction([a,b], d)')
  File "C:\Users\jblasco\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2120, in run_cell_magic
    result = fn(magic_arg_s, cell)
  File "<decorator-gen-61>", line 2, in time
  File "C:\Users\jblasco\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\magic.py", line 193, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "C:\Users\jblasco\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\magics\execution.py", line 1177, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 1, in <module>
  File "C:\Users\jblasco\PycharmProjects\Pr

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



KeyboardInterrupt: 

In [212]:
%%time
c = a & b

Wall time: 634 ms


In [37]:
reload(model)

<module 'model' from 'C:\\Users\\jblasco\\PycharmProjects\\ProcessMining\\app\\module_process_mining\\model.py'>

In [38]:
test_ppm = model.PandasProcessModel("test_ppm", datatable)
ppm = model.PandasProcessModel("ppm", event_log)

In [47]:
test_ppm.get_daily_barchart()

[(1468533600.0, 1.0),
 (1468620000.0, 1.0),
 (1468706400.0, 1.0),
 (1468792800.0, 1.0),
 (1468879200.0, 1.0),
 (1468965600.0, 1.0),
 (1469052000.0, 1.0),
 (1469138400.0, 1.0),
 (1469224800.0, 1.0),
 (1469311200.0, 1.0),
 (1469397600.0, 1.0),
 (1469484000.0, 1.0),
 (1469570400.0, 1.0),
 (1469656800.0, 1.0),
 (1469743200.0, 1.0)]

In [7]:
ppm.get_piechart("user")

(['NOBODY',
  'TISSOTBE',
  'BENEDICTE TISSOT',
  'ALBUQUEL',
  'MANILLAM',
  'NDIAYEIB',
  'CAMERFR',
  'RULOTSY',
  'KANYJA',
  'FRANK CAMER'],
 [52571.0,
  42719.0,
  37043.0,
  22979.0,
  8356.0,
  6643.0,
  4631.0,
  3330.0,
  3219.0,
  2171.0])

In [8]:
filter = model.PandasListFilter("user", 'BENEDICTE TISSOT')

In [9]:
ppm.add_filter(filter)

In [10]:
ppm.get_piechart("user")

(['TISSOTBE',
  'NOBODY',
  'BENEDICTE TISSOT',
  'ALBUQUEL',
  'MANILLAM',
  'RULOTSY',
  'KANYJA',
  'PELLETFA'],
 [38385.0, 37043.0, 37043.0, 20920.0, 7625.0, 3138.0, 2713.0, 1305.0])

In [ ]:
groupby = "user"
value="timestamp"
method="count"

In [ ]:
unsorted_categories, unsorted_values = ppm.get_aggregated_data(groupby, value, method)

In [ ]:
sorted_list = sorted(zip(unsorted_values, unsorted_categories), reverse=True)

In [ ]:
unsort_cat, unsort_val = ppm.get_aggregated_data("user", "timestamp", "count")

In [ ]:
sorted(list(zip(unsort_cat, unsort_val)), key=lambda x: x[1], reverse=True)

In [ ]:
x, y = ppm.get_piechart("user")

In [ ]:
plt.pie(y, labels=x)
plt.show()

In [ ]:
ppm.process_data["event"].unique()

In [ ]:
filtre = model.PandasListFilter('event', 'Manual Release')
ppm.clean_filters()
ppm.add_filter(filtre)

In [ ]:
print(len(ppm.process_data))
print(len(ppm.get_filtered_process_data()))

In [ ]:
x, y = ppm.get_piechart("user")

In [ ]:
plt.pie(y, labels=x)
plt.show()

In [ ]:
ppm.clean_filters()
x, y = ppm.get_piechart("event")

In [ ]:
plt.pie(y, labels=x)
plt.show()

In [ ]:
x, y = ppm.get_daily_barchart()

In [ ]:
plt.bar(x,y)
plt.show()

In [ ]:
import json

In [ ]:
dico = {"bonjour": "hello"}

In [ ]:
test_ppm.get_graph_definition()

In [ ]:
type(test_ppm.get_graph_definition()["graph_edges"][0][2]["case_id"])

In [ ]:
json.dumps(test_ppm.get_daily_barchart())

In [ ]:
json.dumps(test_ppm.get_graph_definition())

In [ ]:
%%time
x, y = ppm.get_daily_barchart("user", "count_unique")

In [ ]:
plt.bar(x,y)
plt.show()

In [ ]:
test_ppm.get_aggregated_data("event", "timestamp", "count")

In [ ]:
np.datetime64(('1970-01-01 00:00:00.000000001'))

In [ ]:
test_ppm.process_data.groupby((test_ppm.process_data["timestamp"]).apply(lambda x: x.date().isocalendar()[1])).agg(np.count_nonzero)

In [ ]:
test_ppm.process_data["timestamp"].iloc[0].date().isocalendar()[1]

In [ ]:
datatable["timestamp"].iloc[0]

In [ ]:
datatable.groupby(datatable["timestam"])

In [ ]:
my_array = np.ndarray((1,2))

In [ ]:
my_array[0][1] = np.datetime64('1970-01-01')

In [ ]:
len([np.datetime64('1970-01-01'), np.datetime64('1970-01-02')])

In [ ]:
%%time
ppm = model.PandasProcessModel('Event log', event_log)

In [ ]:
filtre = model.PandasListFilter('user', 'marchand')

In [ ]:
graph_definition = ppm.get_graph_definition()

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(graph_definition["graph_nodes"])
G.add_edges_from(graph_definition["graph_edges"])

In [ ]:
# pos=graphviz_layout(G,prog='dot')
# nx.draw(G, pos=pos, with_labels=True)
# plt.show()

In [ ]:
def graphml_compatible(G):
    H = G.copy()
    for edge in H.edges(data=True):
        edge[2]["timestamp"] = str(edge[2]["timestamp"])
    return(H)

In [ ]:
nx.write_graphml(G,"event_log_simple_graph.graphml")